# Fetch Top Songs

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

In [114]:
client_id = 'fd79e4c442b84c2495101d0fccb10cb8'
client_secret = '2b40e3b4ea8b4d33b8e531ad2cdb7e14'
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [115]:
token = util.prompt_for_user_token(scope='user-library-read', client_id=client_id, client_secret=client_secret, redirect_uri='http://localhost:8400/')

In [116]:
sp = spotipy.Spotify(auth=token)

In [117]:
import pandas as pd
df = pd.DataFrame(columns=['name', 'album', 'artist', 'id', 'popularity', 'length',
 'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo', 'key',
       'time_signature', 'mood'])

In [118]:
df.columns

Index(['name', 'album', 'artist', 'id', 'popularity', 'length', 'danceability',
       'acousticness', 'energy', 'instrumentalness', 'liveness', 'valence',
       'loudness', 'speechiness', 'tempo', 'key', 'time_signature', 'mood'],
      dtype='object')

In [119]:
playlist_link = "https://open.spotify.com/playlist/2cMXEn09J6ZUqV5m5EQSyC"
playlist_URI = playlist_link.split("/")[-1]
x = sp.playlist_tracks(playlist_URI)["items"][0]
fea = sp.audio_features(x['track']['uri'])[0]
fea

{'danceability': 0.652,
 'energy': 0.698,
 'key': 10,
 'loudness': -4.667,
 'mode': 0,
 'speechiness': 0.042,
 'acousticness': 0.00112,
 'instrumentalness': 0.000115,
 'liveness': 0.0886,
 'valence': 0.47,
 'tempo': 96.021,
 'type': 'audio_features',
 'id': '3E7dfMvvCLUddWissuqMwr',
 'uri': 'spotify:track:3E7dfMvvCLUddWissuqMwr',
 'track_href': 'https://api.spotify.com/v1/tracks/3E7dfMvvCLUddWissuqMwr',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3E7dfMvvCLUddWissuqMwr',
 'duration_ms': 202067,
 'time_signature': 4}

In [120]:
def get_songs(playlist_link,df):
    playlist_URI = playlist_link.split("/")[-1]
    for item in sp.playlist_tracks(playlist_URI)["items"]:
        row = {
        'name':item['track']['name'],
        'album':item['track']['album']['name'],
        'artist':item["track"]["artists"][0]["name"],
        'id':item['track']['id'],
        'popularity':item['track']['popularity'],
        'length':item['track']['duration_ms']
        }
        features = sp.audio_features(item['track']['uri'])[0]
        for key in features:
            if key in df.columns:
                row[key] = features[key]
        df = df.append(row,ignore_index=True)
    return df

In [121]:
links = ['https://open.spotify.com/playlist/6UeSakyzhiEt4NB3UAd6NQ',
'https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF',
'https://open.spotify.com/playlist/37i9dQZF1DWY4lFlS4Pnso']

for link in links:
    df = get_songs(link,df)
    print("Added playlist")

Added playlist
Added playlist
Added playlist


In [126]:
#Remove duplicates
df = df.drop_duplicates()

In [127]:
df.shape

(174, 18)

In [129]:
df.to_csv("latest_songs.csv")

# Quick Train

In [135]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

In [151]:
import pickle
import numpy as np

In [143]:
traindf = pd.read_csv('./SpotifyData/data_moods.csv')
X_train = traindf.loc[:, 'popularity':'time_signature']
X_train['length'] = X_train['length']/max(X_train['length'])

# Mapping class label to respected integer
y_train = traindf['mood'].map({'Happy': 0, 'Sad': 1, 'Energetic': 2, 'Calm':3})
target_names = ['Happy', 'Sad', 'Energetic', 'Calm']

# # Splitting training data and testing data
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

vote_models = []
vote_models.append(('Random Forest Classifier',RandomForestClassifier()))
vote_models.append(('Gradient Boosting Classifier',GradientBoostingClassifier()))
vote_models.append(('XGB Classifier', XGBClassifier()))

model = VotingClassifier(estimators = vote_models, voting ='soft')

model.fit(X_train,y_train)

filename = 'music_model.sav'
pickle.dump(model, open(filename, 'wb'))

# Predict

In [146]:
df = pd.read_csv('latest_songs.csv')

In [144]:
model = pickle.load(open('music_model.sav', 'rb'))

In [152]:
X = df.loc[:,'popularity':'time_signature']
X['length'] = X['length']/max(X['length'])
target_names = ['Happy', 'Sad', 'Energetic', 'Calm']

y = model.predict(X)
labels = {0:'Happy', 1:'Sad', 2:'Energetic', 3:'Calm'}
y_pred = np.vectorize(labels.get)(y)

array(['Happy', 'Happy', 'Happy', 'Sad', 'Sad', 'Energetic', 'Happy',
       'Happy', 'Happy', 'Energetic', 'Happy', 'Happy', 'Happy', 'Sad',
       'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy',
       'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Energetic', 'Happy',
       'Happy', 'Happy', 'Happy', 'Happy', 'Energetic', 'Happy', 'Happy',
       'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy',
       'Happy', 'Happy', 'Happy', 'Happy', 'Calm', 'Happy', 'Energetic',
       'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy',
       'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy',
       'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy',
       'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy',
       'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy',
       'Energetic', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Happy',
       'Happy', 'Happy', 'Sad', 'Energetic', 'Happy', 'Happy', 'Happy',
    

In [154]:
df['mood'] = y_pred

In [157]:
df.to_csv('latest_songs.csv')